In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import time

In [2]:
#df = pd.read_csv('C:/Users/dell/Desktop/Formation Machine Learning engineer/Projet fil rouge/data_cleaned.csv')
df=pd.read_csv('data_2022.csv')
#df = df.drop('Unnamed: 0', axis = 1)



C:\Users\jeanm\AppData\Local\Temp\ipykernel_37292\2335758478.py:2: DtypeWarning: Columns (3,8,10,12,28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data_2022.csv')


In [3]:
df.columns

Index(['ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va',
       'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)',
       'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm',
       'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT', 'Ernedc (g/km)',
       'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
       'Fuel consumption ', 'Electric range (km)'],
      dtype='object')

In [5]:
# Afficher les noms des colonnes catégorielles
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Utilisation de la fonction get_dummies de Pandas pour la dichotomisation
#fuel_dummies = pd.get_dummies(df['Fuel_type'], prefix='Fuel', drop_first=True)
fuel_dummies = pd.get_dummies(df['Ft'], prefix='Fuel', drop_first=True)
df = pd.concat([df, fuel_dummies], axis=1)
#
#country_dummies = pd.get_dummies(df['Pays'], prefix='Country', drop_first=True)
country_dummies = pd.get_dummies(df['Country'], prefix='Country', drop_first=True)
df = pd.concat([df, country_dummies], axis=1)

manufacturer_dummies = pd.get_dummies(df['Mp'], prefix='Manufacturer', drop_first=True)
df = pd.concat([df, manufacturer_dummies], axis=1)
#supprimer les variables 'pays', 'fabriquant' et 'fuel_type' :
#df = df.drop(['Fuel_type','Pays', 'Fabricant'], axis=1)  
df = df.drop(['Ft','Country', 'Mp'], axis=1)  

In [6]:
df.columns

Index(['ID', 'VFN', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va', 'Ve', 'Mk',
       ...
       'Country_SK', 'Manufacturer_FORD', 'Manufacturer_HYUNDAI MOTOR EUROPE',
       'Manufacturer_KIA', 'Manufacturer_MAZDA-SUBARU-SUZUKI-TOYOTA',
       'Manufacturer_MERCEDES-BENZ', 'Manufacturer_RENAULT-NISSAN-MITSUBISHI',
       'Manufacturer_STELLANTIS', 'Manufacturer_TESLA-HONDA-JLR',
       'Manufacturer_VOLKSWAGEN'],
      dtype='object', length=127)

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Charger votre DataFrame (à adapter selon votre source de données)
# df = pd.read_csv('votre_dataset.csv')

# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Créer la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap des corrélations')
plt.show()


ValueError: could not convert string to float: 'IP-07A1A1E024A_000-VF1-1'

### <font color='blue'> scinder le data en deux : </font>
Nous allons decouper le data en deux parties : voitures hybrides et voiture thermiques

In [ ]:
df_hybride = df[df['Consommation_électrique']!=0.0]
df_thermique = df[df['Consommation_électrique']==0.0]

In [ ]:
df_hybride.columns

In [ ]:
df_thermique = df_thermique.drop(columns=['Autonomie_électrique','Consommation_électrique','Fuel_petrol/electric'])
df_thermique.columns

In [ ]:
df_hybride = df_hybride.drop(columns=['Fuel_lpg', 'Fuel_petrol', 'Fuel_petrol/electric'])
df_hybride.columns

### <font color='blue'> Supprimer Consommation electrique et Fuel_petrol/electric : </font>

In [ ]:
df_hybride = df_hybride.drop('Consommation_électrique', axis=1)


In [ ]:
df_hybride.columns

### <font color='blue'> Analyse data voitures thermiques : </font>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Tracer un histogramme
#sns.histplot(df_non_hybride['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black')
sns.histplot(df_thermique['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black', color='blue', line_kws={'color': 'red', 'linewidth': 2})


# Calculer la moyenne et la médiane
mean_value = df_thermique['Emission_co2_(g/km)'].mean()
median_value = df_thermique['Emission_co2_(g/km)'].median()

# Tracer des lignes verticales pour représenter la moyenne et la médiane
plt.axvline(mean_value, color='red', linewidth=1, label='Moyenne de distribution')
plt.axvline(median_value, color='red', linestyle='dashed', linewidth=1, label='Médiane de distribution')

# Ajouter une légende
plt.legend()

# Afficher le graphique
plt.show()


In [ ]:
y = df_thermique['Emission_co2_(g/km)']
X = df_thermique.drop('Emission_co2_(g/km)', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### <font color='blue'> sans validation croisée : </font>

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error



# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Prédiction des valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calcul des métriques pour l'entraînement
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Calcul des métriques pour le test
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Régression Linéaire sans validation croisée")
print(f"R² pour l'entraînement : {r2_train}")
print(f"R² pour le test : {r2_test}")
print(f"MAE pour l'entraînement : {mae_train}")
print(f"MAE pour le test : {mae_test}")
print(f"RMSE pour l'entraînement : {rmse_train}")
print(f"RMSE pour le test : {rmse_test}")


### <font color='blue'> avec validation croisée : </font>

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

# Validation croisée
cv_results = cross_validate(model, X_train_scaled, y_train, cv=10,
                            scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'),
                            return_train_score=True)

# Moyennes des scores R², MAE et RMSE pour l'entraînement et le test
mean_r2_train = np.mean(cv_results['train_r2'])
mean_r2_test = np.mean(cv_results['test_r2'])
mean_mae_train = -np.mean(cv_results['train_neg_mean_absolute_error'])
mean_mae_test = -np.mean(cv_results['test_neg_mean_absolute_error'])
mean_rmse_train = -np.mean(cv_results['train_neg_root_mean_squared_error'])
mean_rmse_test = -np.mean(cv_results['test_neg_root_mean_squared_error'])

print("Régression Linéaire avec validation croisée")
print(f"R² moyen pour l'entraînement : {mean_r2_train}")
print(f"R² moyen pour le test : {mean_r2_test}")
print(f"MAE moyen pour l'entraînement : {mean_mae_train}")
print(f"MAE moyen pour le test : {mean_mae_test}")
print(f"RMSE moyen pour l'entraînement : {mean_rmse_train}")
print(f"RMSE moyen pour le test : {mean_rmse_test}")



### <font color='blue'> analyse des residus : </font>

In [ ]:
# Calculer les résidus
residuals = y_test - y_test_pred

# Visualiser les résidus
plt.figure(figsize=(6, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution des résidus')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.show()
# Enregistrer l'image
plt.savefig('distribution des lr thermiques.png', bbox_inches='tight')

# Vérifier l'homoscédasticité
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_pred, y=residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Résidus par rapport aux valeurs prédites')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.show()

# Enregistrer l'image
plt.savefig('residus par rapport aux valeurs predites lr thermiques.png', bbox_inches='tight')

# Vérifier l'indépendance des résidus
# (Visualisation des résidus par rapport à l'ordre des observations ou à une autre variable)
# Utilisation de tests statistiques pour la normalité des résidus, l'homoscédasticité, etc.


In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Calculer les résidus normalisés
residuals_standardized = (residuals - residuals.mean()) / residuals.std()

# Q-Q plot
plt.figure(figsize=(8, 6))
stats.probplot(residuals_standardized, dist="norm", plot=plt)
plt.title("Q-Q Plot des Résidus")
plt.xlabel("Quantiles théoriques (distribution normale)")
plt.ylabel("Quantiles observés")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculer la moyenne et l'écart type des résidus
residuals_mean = np.mean(residuals)
residuals_std = np.std(residuals)

# Créer le graphique des résidus en fonction des valeurs prédites avec moyenne et écart type
plt.figure(figsize=(10, 6))
plt.scatter(y_train_pred, residuals, alpha=0.5)
plt.title("Graphique des résidus en fonction des valeurs prédites")
plt.xlabel("Valeurs prédites")
plt.ylabel("Résidus")
plt.axhline(y=0, color='crimson', linestyle='--')  # Ajouter une ligne horizontale à y=0 pour référence
plt.axhline(y=residuals_mean, color='crimson', linestyle='-',linewidth=2, label='Moyenne des résidus')  # Ajouter la moyenne des résidus
plt.axhline(y=residuals_mean + 2*residuals_std, color='crimson', linestyle='--', label='±2 σ')  # Ajouter les barres d'écart type
plt.axhline(y=residuals_mean - 2*residuals_std, color='crimson', linestyle='--')
plt.axhline(y=residuals_mean + 3*residuals_std, color='crimson', linestyle='-', label='±3 σ')  # Ajouter les barres d'écart type
plt.axhline(y=residuals_mean - 3*residuals_std, color='crimson', linestyle='-')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Créer le nuage de points entre les valeurs prédites et les valeurs réelles avec une ligne de corrélation
plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, y_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)  # Ajouter une ligne de corrélation parfaite
plt.title("Nuage de points entre les valeurs prédites et les valeurs réelles (test)")
plt.xlabel("Valeurs prédites")
plt.ylabel("Valeurs réelles")
plt.grid(True)
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Définir le style de Seaborn sur darkgrid
sns.set_style("darkgrid")

# Créer la boîte à moustaches des résidus avec Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x=residuals, orient='h', color='cornflowerblue')
plt.title("Boîte à moustaches des résidus")
plt.xlabel("Résidus")
plt.grid(True)
plt.show()



#### <font color='pink'> Elasticnet ridge et lasso : </font>

In [ ]:
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# Initialiser les modèles
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1)

# Entraîner les modèles
elastic_net.fit(X_train_scaled, y_train)
ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

# Prédire les valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred_elastic_net = elastic_net.predict(X_train_scaled)
y_test_pred_elastic_net = elastic_net.predict(X_test_scaled)

y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

y_train_pred_lasso = lasso.predict(X_train_scaled)
y_test_pred_lasso = lasso.predict(X_test_scaled)

# Calculer les scores R² pour l'entraînement et le test
r2_train_elastic_net = r2_score(y_train, y_train_pred_elastic_net)
r2_test_elastic_net = r2_score(y_test, y_test_pred_elastic_net)

r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)

r2_train_lasso = r2_score(y_train, y_train_pred_lasso)
r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

# Calculer les erreurs (MAE et RMSE) pour l'entraînement et le test
mae_train_elastic_net = mean_absolute_error(y_train, y_train_pred_elastic_net)
rmse_train_elastic_net = mean_squared_error(y_train, y_train_pred_elastic_net, squared=False)
mae_test_elastic_net = mean_absolute_error(y_test, y_test_pred_elastic_net)
rmse_test_elastic_net = mean_squared_error(y_test, y_test_pred_elastic_net, squared=False)

mae_train_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
rmse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge, squared=False)
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
rmse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge, squared=False)

mae_train_lasso = mean_absolute_error(y_train, y_train_pred_lasso)
rmse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso, squared=False)
mae_test_lasso = mean_absolute_error(y_test, y_test_pred_lasso)
rmse_test_lasso = mean_squared_error(y_test, y_test_pred_lasso, squared=False)

# Afficher les résultats
print("ElasticNet - R² pour l'entraînement:", r2_train_elastic_net)
print("ElasticNet - R² pour le test:", r2_test_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour le test:", mae_test_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour le test:", rmse_test_elastic_net)

print("\nRidge - R² pour l'entraînement:", r2_train_ridge)
print("Ridge - R² pour le test:", r2_test_ridge)
print("Ridge - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_ridge)
print("Ridge - Mean Absolute Error (MAE) pour le test:", mae_test_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour le test:", rmse_test_ridge)

print("\nLasso - R² pour l'entraînement:", r2_train_lasso)
print("Lasso - R² pour le test:", r2_test_lasso)
print("Lasso - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_lasso)
print("Lasso - Mean Absolute Error (MAE) pour le test:", mae_test_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour le test:", rmse_test_lasso)


### <font color='blue'> Analyse data voitures hybrides : </font>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Tracer un histogramme
sns.histplot(df_hybride['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black', color='blue', line_kws={'color': 'red', 'linewidth': 2})


# Calculer la moyenne et la médiane
mean_value = df_hybride['Emission_co2_(g/km)'].mean()
median_value = df_hybride['Emission_co2_(g/km)'].median()

# Tracer des lignes verticales pour représenter la moyenne et la médiane
plt.axvline(mean_value, color='red', linewidth=1, label='Moyenne de distribution')
plt.axvline(median_value, color='red', linestyle='dashed', linewidth=1, label='Médiane de distribution')

# Ajouter une légende
plt.legend()

# Afficher le graphique
plt.show()


In [ ]:
y = df_hybride['Emission_co2_(g/km)']
X = df_hybride.drop('Emission_co2_(g/km)', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### <font color='blue'> sans validation croisée : </font>

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error



# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Prédiction des valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calcul des métriques pour l'entraînement
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Calcul des métriques pour le test
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Régression Linéaire sans validation croisée")
print(f"R² pour l'entraînement : {r2_train}")
print(f"R² pour le test : {r2_test}")
print(f"MAE pour l'entraînement : {mae_train}")
print(f"MAE pour le test : {mae_test}")
print(f"RMSE pour l'entraînement : {rmse_train}")
print(f"RMSE pour le test : {rmse_test}")


### <font color='blue'> Avec validation croisée : </font>

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

# Validation croisée
cv_results = cross_validate(model, X_train_scaled, y_train, cv=10,
                            scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'),
                            return_train_score=True)

# Moyennes des scores R², MAE et RMSE pour l'entraînement et le test
mean_r2_train = np.mean(cv_results['train_r2'])
mean_r2_test = np.mean(cv_results['test_r2'])
mean_mae_train = -np.mean(cv_results['train_neg_mean_absolute_error'])
mean_mae_test = -np.mean(cv_results['test_neg_mean_absolute_error'])
mean_rmse_train = -np.mean(cv_results['train_neg_root_mean_squared_error'])
mean_rmse_test = -np.mean(cv_results['test_neg_root_mean_squared_error'])

print("Régression Linéaire avec validation croisée")
print(f"R² moyen pour l'entraînement : {mean_r2_train}")
print(f"R² moyen pour le test : {mean_r2_test}")
print(f"MAE moyen pour l'entraînement : {mean_mae_train}")
print(f"MAE moyen pour le test : {mean_mae_test}")
print(f"RMSE moyen pour l'entraînement : {mean_rmse_train}")
print(f"RMSE moyen pour le test : {mean_rmse_test}")


### <font color='blue'> Analyse des résidus : </font>

In [ ]:
# Calculer les résidus
residuals = y_test - y_test_pred

# Visualiser les résidus
plt.figure(figsize=(6, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution des résidus')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.show()
# Enregistrer l'image
plt.savefig('distribution des lr thermiques.png', bbox_inches='tight')

# Vérifier l'homoscédasticité
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_pred, y=residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Résidus par rapport aux valeurs prédites')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.show()


In [ ]:
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# Initialiser les modèles
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1)

# Entraîner les modèles
elastic_net.fit(X_train_scaled, y_train)
ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

# Prédire les valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred_elastic_net = elastic_net.predict(X_train_scaled)
y_test_pred_elastic_net = elastic_net.predict(X_test_scaled)

y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

y_train_pred_lasso = lasso.predict(X_train_scaled)
y_test_pred_lasso = lasso.predict(X_test_scaled)


# Calculer les scores R² pour l'entraînement et le test
r2_train_elastic_net = r2_score(y_train, y_train_pred_elastic_net)
r2_test_elastic_net = r2_score(y_test, y_test_pred_elastic_net)

r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)

r2_train_lasso = r2_score(y_train, y_train_pred_lasso)
r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

# Calculer les erreurs (MAE et RMSE) pour l'entraînement et le test
mae_train_elastic_net = mean_absolute_error(y_train, y_train_pred_elastic_net)
rmse_train_elastic_net = mean_squared_error(y_train, y_train_pred_elastic_net, squared=False)
mae_test_elastic_net = mean_absolute_error(y_test, y_test_pred_elastic_net)
rmse_test_elastic_net = mean_squared_error(y_test, y_test_pred_elastic_net, squared=False)

mae_train_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
rmse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge, squared=False)
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
rmse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge, squared=False)

mae_train_lasso = mean_absolute_error(y_train, y_train_pred_lasso)
rmse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso, squared=False)
mae_test_lasso = mean_absolute_error(y_test, y_test_pred_lasso)
rmse_test_lasso = mean_squared_error(y_test, y_test_pred_lasso, squared=False)

# Afficher les résultats
print("ElasticNet - R² pour l'entraînement:", r2_train_elastic_net)
print("ElasticNet - R² pour le test:", r2_test_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour le test:", mae_test_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour le test:", rmse_test_elastic_net)

print("\nRidge - R² pour l'entraînement:", r2_train_ridge)
print("Ridge - R² pour le test:", r2_test_ridge)
print("Ridge - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_ridge)
print("Ridge - Mean Absolute Error (MAE) pour le test:", mae_test_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour le test:", rmse_test_ridge)

print("\nLasso - R² pour l'entraînement:", r2_train_lasso)
print("Lasso - R² pour le test:", r2_test_lasso)
print("Lasso - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_lasso)
print("Lasso - Mean Absolute Error (MAE) pour le test:", mae_test_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour le test:", rmse_test_lasso)


#### <font color='pink'> Regression linéaire : </font>

In [ ]:
# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Prédiction des valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calcul des métriques de performance
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Affichage des résultats
print(f'R² pour l\'entraînement: {r2_train}')
print(f'R² pour le test: {r2_test}')
print(f'Mean Absolute Error (MAE) pour le test: {mae_test}')
print(f'Root Mean Squared Error (RMSE) pour le test: {rmse_test}')

# Visualisation des résultats
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.title('Nuage de points y_test vs y_test_pred')
plt.xlabel('Valeurs réelles de CO2Emissions')
plt.ylabel('Valeurs prédites de CO2Emissions')
plt.show()

In [ ]:
# Calculer les résidus
residuals = y_test - y_test_pred

# Tracer un graphique des résidus par rapport aux valeurs prédites
plt.scatter(y_test_pred, residuals)
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.title('Graphique des résidus')
plt.axhline(y=0, color='red', linestyle='--')  # Ajouter une ligne à 0 pour la référence
plt.show()

# Histogramme des résidus
plt.hist(residuals, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.title('Histogramme des résidus')
plt.show()

# Graphique quantile-quantile (QQ plot) pour vérifier la normalité des résidus
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Graphique quantile-quantile des résidus')
plt.xlabel('Quantiles théoriques')
plt.ylabel('Quantiles observés')
plt.show()

#### <font color='pink'> Random forest : </font>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Prédictions sur les données de test
y_pred = model.predict(X_test_scaled)



# Evaluation de la performance du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichage des métriques de performance
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur absolue moyenne (MAE) :", mae)
print("Coefficient de détermination (R²) :", r2)

In [ ]:
# Calculer les résidus
residuals = y_test - y_pred

# Tracer un graphique des résidus par rapport aux valeurs prédites
plt.scatter(y_pred, residuals)
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.title('Graphique des résidus')
plt.axhline(y=0, color='red', linestyle='--')  # Ajouter une ligne à 0 pour la référence
plt.show()

# Histogramme des résidus
plt.hist(residuals, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.title('Histogramme des résidus')
plt.show()

# Graphique quantile-quantile (QQ plot) pour vérifier la normalité des résidus
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Graphique quantile-quantile des résidus')
plt.xlabel('Quantiles théoriques')
plt.ylabel('Quantiles observés')
plt.show()

### <font color='blue'> Analyse data complet : </font>


In [ ]:

# Tracer un histogramme
sns.histplot(df['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black', color='blue', line_kws={'color': 'red', 'linewidth': 2})


# Calculer la moyenne et la médiane
mean_value = df['Emission_co2_(g/km)'].mean()
median_value = df['Emission_co2_(g/km)'].median()
# Tracer des lignes verticales pour représenter la moyenne et la médiane
plt.axvline(mean_value, color='red', linewidth=1, label='Moyenne de la distribution')
plt.axvline(median_value, color='red', linestyle='dashed', linewidth=1, label='Médiane de la distribution')

# Ajouter une légende
plt.legend()

# Afficher le graphique
plt.show()


In [ ]:
y = df['Emission_co2_(g/km)']
X = df.drop('Emission_co2_(g/km)', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### <font color='blue'> Régression Linéaire : </font>

La régression linéaire est fondée sur les hypothèses suivantes : centrage, homoscédasticité, indépendance et normalité des erreurs.

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Création du modèle de régression linéaire
model = LinearRegression()

# Entraînement du modèle sur les données d'entraînement
model.fit(X_train_scaled, y_train)


# Prédictions sur les données de test
y_pred = model.predict(X_test_scaled)



# Evaluation de la performance du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichage des métriques de performance
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur absolue moyenne (MAE) :", mae)
print("Coefficient de détermination (R²) :", r2)


**Erreur quadratique moyenne (MSE)** Cela signifie que, en moyenne, les prédictions du modèle diffèrent des vraies valeurs d'environ 142.78 unités au carré.
**Erreur absolue moyenne (MAE)** le MAE est d'environ 8.75. Cela signifie que, en moyenne, les prédictions de votre modèle diffèrent des vraies valeurs d'environ 8.75 unités.
**Coefficient de détermination (R²)** Cela signifie que le modèle explique environ 88% de la variance des émissions de CO2, ce qui est une bonne performance.

**En résumé, notre modèle de régression linéaire semble bien fonctionner, avec un R² élevé et des valeurs de MSE et MAE relativement basses, ce qui indique qu'il est capable de faire des prédictions précises sur les émissions de CO2.**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prédictions sur les données d'entraînement
y_train_pred = model.predict(X_train_scaled)

# Prédictions sur les données de test
y_test_pred = model.predict(X_test_scaled)

# Calcul des métriques sur les données d'entraînement
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Calcul des métriques sur les données de test
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Affichage des métriques pour les données d'entraînement
print("Métriques pour les données d'entraînement :")
print("MSE :", mse_train)
print("MAE :", mae_train)
print("R²  :", r2_train)
print()

# Affichage des métriques pour les données de test
print("Métriques pour les données de test :")
print("MSE :", mse_test)
print("MAE :", mae_test)
print("R²  :", r2_test)


**Interpretation :** Ces métriques indiquent que le modèle a une performance similaire sur les ensembles d'entraînement et de test, ce qui suggère qu'il généralise bien aux données non vues. 
Le R² proche de 0.88 indique que le modèle explique environ 88% de la variance des émissions de CO2, ce qui est une performance solide pour une régression linéaire. 
Les valeurs MSE et MAE relativement faibles indiquent également que les prédictions du modèle sont précises..

#### <font color='pink'> Analyse des résidus : </font>

In [ ]:
# Calculer les résidus
residuals = y_test - y_test_pred

# Tracer un graphique des résidus par rapport aux valeurs prédites
plt.scatter(y_test_pred, residuals)
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.title('Graphique des résidus')
plt.axhline(y=0, color='red', linestyle='--')  # Ajouter une ligne à 0 pour la référence
plt.show()

# Histogramme des résidus
plt.hist(residuals, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.title('Histogramme des résidus')
plt.show()

# Graphique quantile-quantile (QQ plot) pour vérifier la normalité des résidus
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Graphique quantile-quantile des résidus')
plt.xlabel('Quantiles théoriques')
plt.ylabel('Quantiles observés')
plt.show()


In [ ]:
# Afficher les coefficients de régression
coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})
print(coefficients)

In [ ]:
import pandas as pd

# Créer un DataFrame des coefficients de régression
coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})

# Trier les coefficients par valeur absolue
coefficients['Abs_Coefficient'] = abs(coefficients['Coefficient'])
coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False)

# Afficher les coefficients
print(coefficients)


s.

In [ ]:
import matplotlib.pyplot as plt

# Créer un DataFrame des coefficients de régression
coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})

# Trier les coefficients par valeur absolue
coefficients['Abs_Coefficient'] = abs(coefficients['Coefficient'])
coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False)

# Créer le graphique
plt.figure(figsize=(10, 6))
plt.barh(coefficients['Variable'], coefficients['Coefficient'], color='skyblue')
plt.xlabel('Coefficient de régression')
plt.ylabel('Variables')
plt.title('Coefficients de régression')
plt.grid(axis='x')
plt.show()


In [ ]:
# Sélectionner les nouvelles variables explicatives
X_new = df[['Consommation_électrique', 'feature1', 'Puissance_moteur', 'Fuel_petrol/electric', 'Autonomie_électrique']]

# Fractionner les données en ensembles d'entraînement et de test
X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# Standardiser les données
scaler_new = StandardScaler()
X_train_scaled_new = scaler_new.fit_transform(X_train_new)
X_test_scaled_new = scaler_new.transform(X_test_new)

# Créer et entraîner le modèle de régression linéaire
model_new = LinearRegression()
model_new.fit(X_train_scaled_new, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_new = model_new.predict(X_test_scaled_new)

# Calculer les métriques de performance
mse_new = mean_squared_error(y_test, y_pred_new)
mae_new = mean_absolute_error(y_test, y_pred_new)
r2_new = r2_score(y_test, y_pred_new)

# Afficher les métriques de performance
print("Métriques pour les données de test avec les nouvelles variables explicatives :")
print("MSE :", mse_new)
print("MAE :", mae_new)
print("R²  :", r2_new)


**MSE (Mean Squared Error) :**
Nouveau modèle : 167.72
Premier modèle : 142.
**MAE (Mean Absolute Error) :**
Nouveau modèle : 9.69
Premier modèle : 8.
**R² (Coefficient de détermination) :**
Nouveau modèle : 0.858
Premier modèle : 0.8

**En résumé, le premier modèle avec les anciennes variables explicatives semble donner de meilleures performances, avec des valeurs de MSE et de MAE inférieures ainsi qu'un R² plus élevé par rapport au nouveau modèle avec les nouvelles variables explicatives. Cela suggère que les nouvelles variables ne contribuent pas autant à la prédiction de la variable cible par rapport aux anciennes variables. Il pourrait être utile de réexaminer les variables choisies et leur impact sur le modèle.**797578

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

# Définir le modèle de régression linéaire
model = LinearRegression()

# Définir les hyperparamètres à tester
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}

# Créer un objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Exécuter la recherche sur grille sur les données d'entraînement
grid_search.fit(X_train_scaled, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)



In [ ]:
from sklearn.linear_model import LinearRegression

# Créer un nouveau modèle de régression linéaire avec les meilleurs paramètres
best_model = LinearRegression(fit_intercept=True, positive=False)

# Entraîner le modèle sur les données d'entraînement
best_model.fit(X_train_scaled, y_train)

# Faire des prédictions sur les données de test
y_pred_best = best_model.predict(X_test_scaled)

# Calculer les métriques de performance
mse_best = mean_squared_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

# Afficher les métriques de performance
print("Métriques de performance avec les meilleurs paramètres :")
print("MSE :", mse_best)
print("MAE :", mae_best)
print("R²  :", r2_best)


### <font color='blue'> Random forest : </font>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciez le RandomForestRegressor avec les hyperparamètres désirés
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Effectuez une validation croisée pour évaluer les performances du modèle
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse_scores = -cv_scores  # Convertissez les scores de perte négatifs en valeurs positives

# Entraînez le modèle sur les données d'entraînement
rf_model.fit(X_train, y_train)

# Faites des prédictions sur les données de test
y_pred = rf_model.predict(X_test)

# Calculez les métriques de performance
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichez les métriques de performance
print("Scores de validation croisée (MSE) :", cv_mse_scores)
print("Moyenne des scores :", cv_mse_scores.mean())
print("MSE :", mse)
print("MAE :", mae)
print("R²  :", r2)


In [ ]:
import numpy as np
#import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# Créer le modèle de réseau de neurones
model = Sequential([
    Dense(64, input_shape=(10,), activation='relu'),  # Couche cachée avec 64 neurones et fonction d'activation ReLU
    Dense(32, activation='relu'),  # Une autre couche cachée avec 32 neurones et fonction d'activation ReLU
    Dense(1, activation='sigmoid')  # Couche de sortie avec 1 neurone et fonction d'activation sigmoïde pour les problèmes de classification binaire
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


# Définir le modèle CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Ajout de dropout pour prévenir le surapprentissage
    layers.Dense(10)
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

# Évaluer le modèle
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Test accuracy:", test_acc)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models



In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Dropout pour prévenir le surapprentissage
    layers.Dense(num_classes)  # Assurez-vous de remplacer num_classes par le nombre de classes dans vos données
])